In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from main.schlogModel import schloglModel
from main.neuralNetwork import neuralNetworkCME

In [ ]:
# Define Schlogl model
smodel = schloglModel()

# Define and set parameters
concA = 10.
concB = 20.
k1 = 6.
k2 = 1.0
k3 = 230.
k4 = 1000.
vol = 8.
smodel.setModelParameters(concA, concB, k1, k2, k3, k4, vol)

In [ ]:
# Define data parameters
domain = [0, 500]
dt = 0.0001
stride = 10
timesteps = 1000
data_size = 6*4096 #4096 #2560
data_multiplier = 1 #10
extended_data_size = data_size*data_multiplier
in_dimension = 1
out_dimension = 50
smodel.setDataParameters(domain[0], domain[1], dt, stride, timesteps, 
                         data_size, data_multiplier, out_dimension)

In [ ]:
# Generate and/or load data
generateData = False
filename = "data/schloglback_data_vol" + str(vol) + "_ndata" + str(data_size) + ".dat"
if generateData:
    smodel.generateData2(filename)
inputData, targetData = smodel.loadData2(filename)

In [ ]:
# # Generate and/or load data
# generateData = False
# filename = "data/schlogl_data_vol" + str(vol) + "_ndata" + str(data_size) + ".dat"
# if generateData:
#     smodel.generateData()
# inputData, targetData = smodel.loadData(filename)

In [ ]:
# Network setup
neurons_per_layer = 70 #50 #40 #50 #40 #20 #15 #35 #20  # number of neurons per hidden layer
num_layers = 10 #15 #25 #15 #30 #15 #10 #10 #10 #6          # number of hidden layers
activation = tf.nn.relu #tf.sigmoid                     #hidden layer activation function

# training settings
batch_size   = 128 #64 #2048
num_epochs = 200

# Other
outscale = (domain[1] - domain[0])/out_dimension

In [ ]:
# Define the neural network
nnetworkCME = neuralNetworkCME(inputData, targetData, num_layers, neurons_per_layer, 
                               activation, in_dimension, out_dimension, smodel)

In [ ]:
# Run the training
nnetworkCME.train(10, batch_size, extended_data_size)

In [ ]:
# Plor for vector case
plt.rcParams['figure.figsize'] = (11,8)

# Generate full sample grid of input domain
xgrid = np.linspace(domain[0] + 1, domain[1], num=500)
fwdtime = 0
time = timesteps/stride - fwdtime*np.ones(len(xgrid))
input_test_batch = np.reshape(np.column_stack((xgrid,time)) , [-1,2])

# Generate output with network
prediction_test_batch = nnetworkCME.runNetwork(input_test_batch)

# Remove negative entries and renormalize
x0index = 350
#renormalized_output = np.mean(prediction_test_batch, axis=0)
renormalized_output = prediction_test_batch[x0index]
renormalized_output[renormalized_output<0] = 0.0
renormalized_output = renormalized_output/np.sum(renormalized_output)

plt.bar(np.arange(outscale/2,outscale*out_dimension,outscale),renormalized_output/outscale, 
        width=outscale, label="NN", color=(0.0, 0.4, 1.0, 0.5))

# Calculate steady state analytically
n=np.linspace(domain[0], (domain[1]-domain[0])-1, domain[1]-domain[0])
ss_solution=np.zeros(len(n))
for i in range(len(n)):
    ss_solution[i] = smodel.steadystate_solution(i)

ss_solution = ss_solution/np.sum(ss_solution)

# Plot analytic solution
plt.plot(n,ss_solution, '-r', lw = 3, label="Steady state (exact)")
# plt.plot(n,np.log(ss_solution))

plt.xlim([0, 400])
#plt.ylim([0.0,0.02])
plt.ylabel('Probability', fontsize = 35)
plt.xlabel('$X[T]$', fontsize = 35)
plt.legend(fontsize = 35)
plt.tick_params(labelsize=30)
plt.locator_params(axis='y', nbins=6)
plt.show()

In [ ]:
def interactive_output(x,t):
    netInput = np.reshape(np.column_stack((x,t)) , [-1,2]) 
    prediction = session.run( networkPrediction, feed_dict={networkInput: netInput})
    # Remove negative entries and renormalize
    #renormalized_output = np.mean(prediction_test_batch, axis=0)
    prediction[prediction<0] = 0.0
    prediction = prediction/np.sum(prediction)

    plt.bar(np.arange(1,outscale*OUTPUT_RES,outscale),prediction[0]/outscale, width=outscale)
    plt.plot(n,ss_solution, '-r')
    plt.xlim([DOMAIN[0], DOMAIN[1]-100])
    plt.ylim([0, 0.05])
    plt.ylabel('Histogram')
    plt.show()

interactive_plot = interactive(interactive_output, x=(DOMAIN[0],DOMAIN[1]-100,10), t=(0,100,10))
output = interactive_plot.children[-1]
interactive_plot

## TESTS

In [ ]:
a=5
b=3
np.stack([a,b])

In [ ]:
# Define intensity functions
def lambda1(n):
    return concA*k1*n*(n-1)/vol
def lambda2(n):
    return k2*n*(n-1)*(n-2)/vol**2
def lambda3(n):
    return concB*k3*vol + 0*n
def lambda4(n):
    return  n*k4

def diffusion(x):
    ''' Define diffusion of corresponding Chemical Langevin equation (CLE)'''
    sig = [tf.sqrt(lambda1(x)), -1* tf.sqrt(lambda2(x)), tf.sqrt(lambda3(x)), -1* tf.sqrt(lambda4(x))]
    sigma = tf.stack(sig)
    return sigma
    
def sigma(x):
    ''' Define matrix in Chemical Fokker-Planck equation (CFPE), returns as tensor flow object'''
    diff = diffusion(x)
    return tf.tensordot(diff, diff, axes=2) #np.tensordot(diff,diff.transpose(), 0)

In [ ]:
a = tf.constant([7.0], shape=[1])
diff = diffusion(a)
bb = sigma(a)

In [ ]:
#a = tf.constant([7.0], shape=[1, 1])
#func = diff
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
with tf.Session() as sess:
    #a.initializer.run()
    #func.initializer.run()
    #result=func.eval()
    #result=func.eval()
    print(session.run(diff))
    print(session.run(bb))
#result

In [ ]:
17.74**2

In [ ]:
a = np.array([ 17.74824, -1.8114221, 191.83327, -83.666    ])

In [ ]:
lambda1(7)+lambda2(7)+lambda3(7)+lambda4(7)

In [ ]:
a=smodel.diffusion(7)

In [ ]:
np.tensordot(a,a.transpose(), 1)